# Model Optimization

## Fine-tuning
Here we have some examples using the OpenAI API. Instead, it could have been done using the OpenAI dashboard (easier).

### Resposta que gostaríamos do modelo

```
{
  "resposta": "Equação quadrática é uma equação polinomial de segundo grau, com a forma ax² + bx + c = 0.",
  "categoria": "Matemática",
  "fonte": "AsimoBot"
}
```

### Resposta que o modelo está nos dando

Uma equação quadrática é uma equação polinomial de segundo grau, ou seja, uma equação na forma ax^2 + bx + c = 0, onde a, b e c são constantes e a é diferente de zero. A incógnita da equação é x e o objetivo é encontrar os valores de x que satisfazem a equação. As equações quadráticas podem ter duas soluções reais, uma solução real ou duas soluções complexas.

### Prompt para que ele nos dê a mensagem formatada como queremos

system_mes = '''
Responda as perguntas em um parágrafo de até 20 palavras. Categorize as respostas no seguintes conteúdos: física, matemática, língua portuguesa ou outros.
Retorne a resposta em um formato json, com as keys: 
fonte: valor deve ser sempre AsimoBot
resposta: a resposta para a pergunta
categoria: a categoria da pergunta
'''

### Formatação das mensagens para realizarmos o Fine Tuning

```
{"messages": 
    [
        {"role": "user", "content": "O que é uma equação quadrática"},
        {"role": "assistant", "content": 
            {
                "resposta": "Equação quadrática é uma equação polinomial de segundo grau, com a forma ax² + bx + c = 0.",
                "categoria": "Matemática",
                "fonte": "AsimoBot"
            }},
    ]
}

```

### Criando o arquivo JSONL

In [4]:
import json

with open('arquivos/chatbot_respostas.json', encoding="utf8") as f:
    json_respostas = json.load(f)

In [9]:
with open('arquivos/chatbot_respostas.jsonl', 'w', encoding="utf8") as f:
    for entrada in json_respostas:
        resposta = {
            'resposta': entrada['resposta'],
            'categoria': entrada['categoria'],
            'fonte': 'AsimoBot'
        }
        entrada_jsonl = {
            'messages': [
                {'role': 'user', 'content': entrada['pergunta']},
                {'role': 'assistant', 'content': json.dumps(resposta, ensure_ascii=False, indent=2)}
            ]
        }
        json.dump(entrada_jsonl, f, ensure_ascii=False)
        f.write('\n')

In [10]:
import openai
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

client = openai.Client()

In [12]:
file = client.files.create(
    file=open('arquivos/chatbot_respostas.jsonl',  'rb'),
    purpose='fine-tune'
)

client.fine_tuning.jobs.create(
    training_file=file.id,
    model="gpt-3.5-turbo"
)

FineTuningJob(id='ftjob-8B9yv1osCAHuvt4Sd5afrXj5', created_at=1711459948, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-C4LP6dVchevKWuzzOjSndXmY', result_files=[], status='validating_files', trained_tokens=None, training_file='file-YJ9Hbl9XhbEW4l14g47ck5RW', validation_file=None, user_provided_suffix=None)

In [13]:
client.fine_tuning.jobs.list()

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-8B9yv1osCAHuvt4Sd5afrXj5', created_at=1711459948, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-C4LP6dVchevKWuzzOjSndXmY', result_files=[], status='validating_files', trained_tokens=None, training_file='file-YJ9Hbl9XhbEW4l14g47ck5RW', validation_file=None, user_provided_suffix=None), FineTuningJob(id='ftjob-OOkYNLkBLuQV7S8ki9kAMQlV', created_at=1710789233, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model='ft:gpt-3.5-turbo-0125:personal::94CwIN1U', finished_at=1710790077, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-C4LP6dVchevKWuzzOjSndXmY', result_files=['file-Q3JbaF

### Utilizando o Modelo

In [14]:
mensagens = [{'role': 'user', 'content': 'O que é uma equação quadrática?'}]

resposta = client.chat.completions.create(
    messages=mensagens,
    model="gpt-3.5-turbo",
    max_tokens=1000,
    temperature=0
)

print(resposta.choices[0].message.content)

Uma equação quadrática é uma equação polinomial de segundo grau, ou seja, uma equação na forma ax^2 + bx + c = 0, onde a, b e c são constantes e a ≠ 0. A incógnita da equação é representada por x e o objetivo é encontrar os valores de x que satisfazem a equação. As equações quadráticas podem ter duas soluções reais, uma solução real ou duas soluções complexas, dependendo do valor do discriminante (b^2 - 4ac).


In [15]:
system_mes = '''
Responda as perguntas em um parágrafo de até 20 palavras. Categorize as respostas no seguintes conteúdos: física, matemática, língua portuguesa ou outros.
Retorne a resposta em um formato json, com as keys: 
fonte: valor deve ser sempre AsimoBot
resposta: a resposta para a pergunta
categoria: a categoria da pergunta
'''

mensagens = [
    {'role': 'system', 'content': system_mes},
    {'role': 'user', 'content': 'O que é uma equação quadrática?'}
    ]

resposta = client.chat.completions.create(
    messages=mensagens,
    model="gpt-3.5-turbo",
    max_tokens=1000,
    temperature=0
)

print(resposta.choices[0].message.content)

{
    "fonte": "AsimoBot",
    "resposta": "Uma equação quadrática é uma equação do segundo grau, ou seja, uma equação que possui uma incógnita elevada ao quadrado.",
    "categoria": "matemática"
}


In [17]:
resposta.usage

CompletionUsage(completion_tokens=61, prompt_tokens=114, total_tokens=175)

In [18]:
mensagens = [
    {'role': 'user', 'content': 'O que é uma equação quadrática?'}
    ]

resposta = client.chat.completions.create(
    messages=mensagens,
    model="ft:gpt-3.5-turbo-0125:personal::971R86c5",
    max_tokens=1000,
    temperature=0
)

print(resposta.choices[0].message.content)

{
  "resposta": "Uma equação quadrática é uma equação polinomial de segundo grau, com a forma ax² + bx + c = 0.",
  "categoria": "Matemática",
  "fonte": "AsimoBot"
}


In [19]:
resposta.usage

CompletionUsage(completion_tokens=58, prompt_tokens=16, total_tokens=74)